In [1]:
import torch

x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

In [ ]:
# y = x.T @ x
x.requires_grad_(True)  # 等价于x=torch.arange(4.0,requires_grad=True)
print(x.grad)  # 默认值是None

tensor([ 0.,  4.,  8., 12.])


In [4]:
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

In [5]:
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

In [7]:
# y_grad == 4 * x
x.grad == 4 * x

tensor([True, True, True, True])

In [8]:
# 在默认情况下，PyTorch会累积梯度，我们需要清除之前的值
x.grad.zero_()
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

In [9]:
# 2.5.2. 非标量变量的反向传播
# 向量y关于向量x的导数的最自然解释是一个矩阵。 对于高阶和高维的y和x，求导的结果可以是一个高阶张量。
# 虽然这些更奇特的对象确实出现在高级机器学习中（包括深度学习中）， 但当调用向量的反向计算时，
# 我们通常会试图计算一批训练样本中每个组成部分的损失函数的导数。 
# 这里，我们的目的不是计算微分矩阵，而是单独计算批量中每个样本的偏导数之和。

# 对非标量调用backward需要传入一个gradient参数，该参数指定微分函数关于self的梯度。
# 本例只想求偏导数的和，所以传递一个1的梯度是合适的
x.grad.zero_()
y = x * x
# 等价于y.backward(torch.ones(len(x)))
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

In [10]:
# 2.5.3. 分离计算
# 有时，我们希望将某些计算移动到记录的计算图之外。 例如，假设y是作为x的函数计算的，而z则是作为y和x的函数计算的。 
# 想象一下，我们想计算z关于x的梯度，但由于某种原因，希望将y视为一个常数， 并且只考虑到x在y被计算后发挥的作用。
# 这里可以分离y来返回一个新变量u，该变量与y具有相同的值， 但丢弃计算图中如何计算y的任何信息。 
# 换句话说，梯度不会向后流经u到x。 因此，下面的反向传播函数计算z=u*x关于x的偏导数，
# 同时将u作为常数处理， 而不是z=x*x*x关于x的偏导数。

x.grad.zero_()
y = x * x
u = y.detach() # 分离梯度
z = u * x

z.sum().backward()
x.grad == u

tensor([True, True, True, True])

In [11]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

In [12]:
# 2.5.4. Python控制流的梯度计算
# 使用自动微分的一个好处是： 即使构建函数的计算图需要通过Python控制流
# （例如，条件、循环或任意函数调用），我们仍然可以计算得到的变量的梯度。 
# 在下面的代码中，while循环的迭代次数和if语句的结果都取决于输入a的值。

def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

In [13]:
a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()

In [14]:
# 我们现在可以分析上面定义的f函数。 请注意，它在其输入a中是分段线性的。 
# (a)=k*a，其中k的值取决于输入a，因此可以用d/a验证梯度是否正确。
a.grad == d / a

tensor(True)